In [16]:
import sys
sys.path.append("../src")

import generate_general_networks as generate
import torch
import matplotlib.pyplot as plt
import json
import numpy as np
import tikzplotlib
import loading_json as load
import json

In [17]:
# Load results from optimization
f = open("../optimization_results/general_optimization/single_lane_1.json")
results = json.load(f)
f.close()

In [18]:
# Collecting network configuration
network_file = results['network_file']
f = open("../"+network_file)
network_config = json.load(f)
f.close()

T = network_config['T']
T = 100
N = network_config['N']
controls = network_config['control_points'][0]

In [19]:
# Collecting the start and final parameters
start = results['parameters'][0]
end = results['parameters'][-1]

In [20]:
# Create the networks
start_speed = [torch.tensor(v) for v in start]
end_speed = [torch.tensor(v) for v in end]

start_network = generate.single_lane_network(T, N, start_speed, controls, track_grad=False)
end_network = generate.single_lane_network(T, N, end_speed, controls, track_grad=False)

In [21]:
# Do the simulations
densities, _, lengths, delays, n_stops = start_network.solve_cons_law_counting()
end_densities, _, end_lengths, end_delays, end_n_stops = end_network.solve_cons_law_counting()

Bus bus_1 reached bus stop 0 at time 51.428550720214844, should wait for 29.830259323120117 seconds
Bus bus_1 reached bus stop 0 at time 20.571435928344727, should wait for 29.932104110717773 seconds


In [22]:
# Store densities
densities_loaded = load.convert_from_tensor(densities)
bus_lengths_loaded = load.convert_from_tensor(lengths)

end_densities_loaded = load.convert_from_tensor(end_densities)
end_bus_lengths_loaded = load.convert_from_tensor(end_lengths)

In [23]:
with open("../general_densities/single_lane_start.json", 'w') as fd:
    fd.write(json.dumps([densities_loaded, bus_lengths_loaded]))

In [24]:
with open("../general_densities/single_lane_optimal.json", 'w') as fd:
    fd.write(json.dumps([end_densities_loaded, end_bus_lengths_loaded]))

In [25]:
# Convert to the same time scale:
# Use single_lane_optimal as the goal


f = open("../general_densities/single_lane_start.json")
data = json.load(f)
f.close()
orig_densities = data[0]
orig_lengths = data[1]

f = open("../general_densities/single_lane_optimal.json")
data_goal = json.load(f)
f.close()
goal_densities = data_goal[0]

In [26]:
goal_times = list(goal_densities['0'].keys())
goal_times = [float(t) for t in goal_times]

old_times = list(orig_densities['0'].keys())
old_times = [float(t) for t in old_times]

In [27]:
new_lengths = {i : {} for i in orig_lengths.keys()}
new_densities = {i : {} for i in orig_densities.keys()}

prev_idx = 0
for t in goal_times:
    add_incr = 0
    for t_prev, t_next in zip(old_times[prev_idx:-1], old_times[prev_idx+1:]):
        if t_prev <= t and t_next >= t:
            prev_dist = t - t_prev
            next_dist = t_next - t
            interval_length = t_next - t_prev
            prev_weight = (interval_length - prev_dist) / interval_length
            next_weight = (interval_length - next_dist) / interval_length
            for bus_id in orig_lengths.keys(): 
                new_lengths[bus_id][str(t)] = orig_lengths[bus_id][str(t_prev)]*prev_weight + orig_lengths[bus_id][str(t_next)]*next_weight
            for road_id in orig_densities.keys():
                new_densities[road_id][str(t)] = [orig_densities[road_id][str(t_prev)][i]*prev_weight + orig_densities[road_id][str(t_next)][i]*next_weight for i in range(len(orig_densities[road_id][str(t_prev)]))]
            break
        else:
            add_incr += 1

In [28]:
# Write to file
with open("../general_densities/single_lane_start_opt_times.json", 'w') as fd:
    fd.write(json.dumps([new_densities, new_lengths]))